## Import Libraries

In [4]:
!pip install seaborn  

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [6]:
!pip install torch 

  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/74.5 MB ? eta -:--:--Downloading torch-2.9.1-cp310-none-macosx_11_0_arm64.whl (74.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 3.5 MB/s eta 0:00:0000:0100:010m
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 3.5 MB/s eta 0:00:0000:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached filelock-3.20.0-py3-none-any.whl (16 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached filelock-3.20.0-py3-none-any.whl (16 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (5

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Add parent directory to path to import model
sys.path.insert(0, os.path.abspath('..'))
from src.models.cnn import SeismicCNN, CompactSeismicCNN

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("\n✓ Libraries imported successfully!")

Using device: cpu

✓ Libraries imported successfully!


## Load Labeled Data

Load the windowed seismograms and labels created by the multi-class labeling notebook.

In [8]:
# Find the most recent labeled dataset
data_dir = Path("labeled_data")

if not data_dir.exists():
    raise FileNotFoundError(f"Data directory '{data_dir}' not found. Run multi_class_labeling.ipynb first.")

# Find most recent files
waveform_files = sorted(data_dir.glob("windowed_waveforms_*.npy"))
label_files = sorted(data_dir.glob("labels_*.npy"))
metadata_files = sorted(data_dir.glob("metadata_*.csv"))

if not waveform_files or not label_files:
    raise FileNotFoundError("No labeled data found. Run multi_class_labeling.ipynb first.")

# Use most recent files
waveforms_file = waveform_files[-1]
labels_file = label_files[-1]
metadata_file = metadata_files[-1] if metadata_files else None

print(f"Loading data from:")
print(f"  Waveforms: {waveforms_file.name}")
print(f"  Labels: {labels_file.name}")
if metadata_file:
    print(f"  Metadata: {metadata_file.name}")

# Load data
X = np.load(waveforms_file)  # Shape: (n_samples, window_length)
y = np.load(labels_file)     # Shape: (n_samples,)

if metadata_file:
    metadata = pd.read_csv(metadata_file)
else:
    metadata = None

print(f"\n✓ Data loaded successfully!")
print(f"  X shape: {X.shape}")
print(f"  y shape: {y.shape}")

# Print class distribution
class_names = ['Noise', 'Traffic', 'Earthquake']
unique, counts = np.unique(y, return_counts=True)
print(f"\nClass distribution:")
for label, count in zip(unique, counts):
    print(f"  {class_names[int(label)]} (class {int(label)}): {count} samples ({count/len(y)*100:.1f}%)")

Loading data from:
  Waveforms: windowed_waveforms_20251210_170458.npy
  Labels: labels_20251210_170458.npy
  Metadata: metadata_20251210_170458.csv

✓ Data loaded successfully!
  X shape: (719, 500)
  y shape: (719,)

Class distribution:
  Noise (class 0): 300 samples (41.7%)
  Traffic (class 1): 18 samples (2.5%)
  Earthquake (class 2): 401 samples (55.8%)


## Prepare Data for Training

Split data into train/validation/test sets and create PyTorch DataLoaders.

In [9]:
# Add channel dimension for CNN: (n_samples, window_length) -> (n_samples, 1, window_length)
X = X[:, np.newaxis, :]  # Add channel dimension

print(f"Reshaped X: {X.shape}")

# Split data: 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"\nData split:")
print(f"  Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Val:   {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Print class distribution per split
for split_name, y_split in [("Train", y_train), ("Val", y_val), ("Test", y_test)]:
    print(f"\n{split_name} class distribution:")
    unique, counts = np.unique(y_split, return_counts=True)
    for label, count in zip(unique, counts):
        print(f"  {class_names[int(label)]}: {count} ({count/len(y_split)*100:.1f}%)")

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\n✓ DataLoaders created with batch size: {batch_size}")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

Reshaped X: (719, 1, 500)

Data split:
  Train: 503 samples (70.0%)
  Val:   108 samples (15.0%)
  Test:  108 samples (15.0%)

Train class distribution:
  Noise: 210 (41.7%)
  Traffic: 13 (2.6%)
  Earthquake: 280 (55.7%)

Val class distribution:
  Noise: 45 (41.7%)
  Traffic: 2 (1.9%)
  Earthquake: 61 (56.5%)

Test class distribution:
  Noise: 45 (41.7%)
  Traffic: 3 (2.8%)
  Earthquake: 60 (55.6%)

✓ DataLoaders created with batch size: 32
  Train batches: 16
  Val batches: 4
  Test batches: 4


## Initialize Model

Create the CNN model for 3-class classification.

In [10]:
# Model configuration
num_classes = 3
input_channels = 1  # Single channel (vertical component)
input_length = X_train.shape[2]  # Window length in samples

print(f"Model configuration:")
print(f"  Input channels: {input_channels}")
print(f"  Input length: {input_length} samples")
print(f"  Number of classes: {num_classes}")

# Choose model type: 'standard' or 'compact'
model_type = 'compact'  # Use compact model for faster training

if model_type == 'standard':
    model = SeismicCNN(
        num_classes=num_classes,
        input_channels=input_channels,
        input_length=input_length,
        dropout_rate=0.3
    )
else:
    model = CompactSeismicCNN(
        num_classes=num_classes,
        input_channels=input_channels,
        input_length=input_length
    )

model = model.to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n✓ {model_type.capitalize()} model created")
print(f"  Total trainable parameters: {n_params:,}")

# Print model architecture
print(f"\nModel architecture:")
print(model)

Model configuration:
  Input channels: 1
  Input length: 500 samples
  Number of classes: 3

✓ Compact model created
  Total trainable parameters: 9,347

Model architecture:
CompactSeismicCNN(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (global_pool): AdaptiveAvgPool1d(output_size=1)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)


## Training Configuration

Set up loss function, optimizer, and training parameters.

In [ ]:
# Training hyperparameters
num_epochs = 50
learning_rate = 0.001
weight_decay = 1e-4

# Loss function (with class weights for imbalanced data)
class_counts = np.bincount(y_train)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_weights)  # Normalize
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Learning rate scheduler - simple step decay every 15 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

print(f"Training configuration:")
print(f"  Epochs: {num_epochs}")
print(f"  Learning rate: {learning_rate}")
print(f"  LR decay: 0.5x every 15 epochs")
print(f"  Weight decay: {weight_decay}")
print(f"  Batch size: {batch_size}")
print(f"\nClass weights:")
for i, (name, weight) in enumerate(zip(class_names, class_weights)):
    print(f"  {name}: {weight:.3f}")
print(f"\n✓ Training configuration complete")

TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

## Training Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train for one epoch.
    
    Returns:
        avg_loss: Average training loss
        accuracy: Training accuracy
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / total
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy


def validate_epoch(model, val_loader, criterion, device):
    """
    Validate for one epoch.
    
    Returns:
        avg_loss: Average validation loss
        accuracy: Validation accuracy
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / total
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy


print("✓ Training functions defined")

## Train Model

Train the model and track training/validation metrics.

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_loss = float('inf')
best_model_state = None

print(f"Starting training for {num_epochs} epochs...\n")
print(f"{'Epoch':<6} {'Train Loss':<12} {'Train Acc':<12} {'Val Loss':<12} {'Val Acc':<12} {'Time':<8}")
print("-" * 70)

import time

for epoch in range(num_epochs):
    start_time = time.time()
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate (call after each epoch)
    scheduler.step()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
    
    # Print progress
    epoch_time = time.time() - start_time
    print(f"{epoch+1:<6} {train_loss:<12.4f} {train_acc:<12.2f} {val_loss:<12.4f} {val_acc:<12.2f} {epoch_time:<8.2f}s")

print("\n" + "="*70)
print(f"✓ Training complete!")
print(f"  Best validation loss: {best_val_loss:.4f}")
print(f"  Final train accuracy: {history['train_acc'][-1]:.2f}%")
print(f"  Final val accuracy: {history['val_acc'][-1]:.2f}%")

# Load best model
model.load_state_dict(best_model_state)
print(f"\n✓ Best model loaded")

## Visualize Training History

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Training curves plotted")

## Evaluate on Test Set

In [ ]:
# Evaluate on test set
model.eval()
y_true = []
y_pred = []
y_proba = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs.data, 1)
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        y_proba.extend(probabilities.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_proba = np.array(y_proba)

# Calculate metrics
test_accuracy = accuracy_score(y_true, y_pred)

print(f"Test Set Results:")
print(f"  Overall Accuracy: {test_accuracy*100:.2f}%")
print(f"\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=3))

## Confusion Matrix

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, 
            yticklabels=class_names, ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('True', fontsize=12)
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14)

# Normalized
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names,
            yticklabels=class_names, ax=axes[1], cbar_kws={'label': 'Proportion'})
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('True', fontsize=12)
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14)

plt.tight_layout()
plt.show()

print("✓ Confusion matrix plotted")

## Visualize Predictions on Test Examples

In [ ]:
# Select random examples from test set
n_examples = 9
random_indices = np.random.choice(len(X_test), n_examples, replace=False)

fig, axes = plt.subplots(3, 3, figsize=(15, 10))
axes = axes.flatten()

for i, idx in enumerate(random_indices):
    ax = axes[i]
    
    # Get waveform
    waveform = X_test[idx, 0, :]  # Remove channel dimension
    true_label = y_test[idx]
    pred_label = y_pred[idx]
    proba = y_proba[idx]
    
    # Plot waveform
    time_axis = np.arange(len(waveform)) / 100.0  # Assuming 100 Hz
    color = 'green' if true_label == pred_label else 'red'
    ax.plot(time_axis, waveform, color=color, linewidth=1)
    
    # Title with prediction info
    title = f"True: {class_names[true_label]}, Pred: {class_names[pred_label]}\n"
    title += f"Conf: {proba[pred_label]*100:.1f}%"
    ax.set_title(title, fontsize=10, color=color)
    ax.set_xlabel('Time (s)', fontsize=9)
    ax.set_ylabel('Amplitude', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.suptitle('Test Set Predictions (Green=Correct, Red=Incorrect)', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

print("✓ Example predictions visualized")

## Save Trained Model

In [ ]:
# Create models directory
models_dir = Path("../models")
models_dir.mkdir(exist_ok=True)

# Save model
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"seismic_cnn_{model_type}_{timestamp}.pth"
model_path = models_dir / model_filename

# Save model state dict and metadata
torch.save({
    'model_state_dict': model.state_dict(),
    'model_type': model_type,
    'num_classes': num_classes,
    'input_channels': input_channels,
    'input_length': input_length,
    'class_names': class_names,
    'test_accuracy': test_accuracy,
    'history': history,
    'training_config': {
        'num_epochs': num_epochs,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'weight_decay': weight_decay
    }
}, model_path)

print(f"✓ Model saved to: {model_path}")
print(f"\nModel info:")
print(f"  Type: {model_type}")
print(f"  Parameters: {n_params:,}")
print(f"  Test accuracy: {test_accuracy*100:.2f}%")
print(f"  Input shape: (batch, {input_channels}, {input_length})")
print(f"  Output classes: {num_classes}")

# Save training summary
summary_file = models_dir / f"training_summary_{timestamp}.txt"
with open(summary_file, 'w') as f:
    f.write("Seismic CNN Training Summary\n")
    f.write("="*60 + "\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("Model Configuration:\n")
    f.write(f"  Type: {model_type}\n")
    f.write(f"  Parameters: {n_params:,}\n")
    f.write(f"  Input shape: (batch, {input_channels}, {input_length})\n")
    f.write(f"  Number of classes: {num_classes}\n\n")
    
    f.write("Training Configuration:\n")
    f.write(f"  Epochs: {num_epochs}\n")
    f.write(f"  Batch size: {batch_size}\n")
    f.write(f"  Learning rate: {learning_rate}\n")
    f.write(f"  Weight decay: {weight_decay}\n\n")
    
    f.write("Dataset Split:\n")
    f.write(f"  Train: {len(X_train)} samples\n")
    f.write(f"  Validation: {len(X_val)} samples\n")
    f.write(f"  Test: {len(X_test)} samples\n\n")
    
    f.write("Results:\n")
    f.write(f"  Best validation loss: {best_val_loss:.4f}\n")
    f.write(f"  Final train accuracy: {history['train_acc'][-1]:.2f}%\n")
    f.write(f"  Final val accuracy: {history['val_acc'][-1]:.2f}%\n")
    f.write(f"  Test accuracy: {test_accuracy*100:.2f}%\n\n")
    
    f.write("Classification Report:\n")
    f.write(classification_report(y_true, y_pred, target_names=class_names, digits=3))

print(f"\n✓ Training summary saved to: {summary_file}")
print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)